In [2]:
OPENAI_KEY = "OPENAI_KEY"

In [3]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains.qa_generation.base import QAGenerationChain
from langchain.chains import LLMChain, ConversationalRetrievalChain, RetrievalQAWithSourcesChain
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.retrievers import MultiQueryRetriever
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI
from langchain.agents import AgentType, OpenAIFunctionsAgent, ZeroShotAgent, initialize_agent
from langchain.utilities import TextRequestsWrapper, DuckDuckGoSearchAPIWrapper
from langchain import WikipediaAPIWrapper
from langchain.tools import WikipediaQueryRun, RequestsPostTool
from langchain.agents.tools import Tool
from langchain.tools.ddg_search.tool import DuckDuckGoSearchRun
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent, create_retriever_tool
from langchain.document_transformers import DoctranPropertyExtractor
from langchain.document_transformers import LongContextReorder
from langchain.document_loaders import PyPDFLoader, WebBaseLoader
from langchain.text_splitter import LatexTextSplitter, RecursiveCharacterTextSplitter, TokenTextSplitter, NLTKTextSplitter, CharacterTextSplitter
from bs4 import BeautifulSoup

In [4]:
ddg_api = DuckDuckGoSearchAPIWrapper()

In [5]:
pdf_loader = PyPDFLoader(file_path='data/Сергей Братчиков, CV.pdf')
pdf_docs = pdf_loader.load()
len(pdf_docs)

2

In [6]:
web_loader = WebBaseLoader(web_path="http://ria.ru/20230809/yaponiya-1888975642.html")
web_docs = web_loader.load()
len(web_docs)

1

In [11]:
wiki_api = WikipediaAPIWrapper()

In [12]:
llm = ChatOpenAI(model_name='gpt-3.5-turbo-16k', openai_api_key=OPENAI_KEY)

In [13]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_KEY)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

In [14]:
all_splits = text_splitter.split_documents(pdf_docs)
len(all_splits)

11

In [15]:
vectorstore = Chroma.from_documents(documents=all_splits, embedding=embeddings, persist_directory='data/chroma')
# vectorstore.persist()

In [16]:
all_splits[0]

Document(page_content='Братчиков Сергей Станиславович ♂phone+79859856098\nРоль: ML Engineer /envel⌢pehivaze.me@gmail.com\n31.10.2000 (22 года) /githubGitHub Profile\n/linkedinLinkedIn Profile\nОбразование\n•Бакалавриат: Национальный исследовательский технологический университет МИСиС 2018-2023\nНаправление: Прикладная математика GPA: 4.1/5.0\nТема диплома: Динамическая оптимизация инвестиционного портфеля с Reinforcement Learning\nОпыт работы\n•Pheon Digital Twins Дек 2022 - Сейчас\nNLP Engineer International', metadata={'source': 'data/Сергей Братчиков, CV.pdf', 'page': 0})

In [17]:
vectorstore.similarity_search_with_score("Имя и фамилия", k=3)

[(Document(page_content='Братчиков Сергей Станиславович ♂phone+79859856098\nРоль: ML Engineer /envel⌢pehivaze.me@gmail.com\n31.10.2000 (22 года) /githubGitHub Profile\n/linkedinLinkedIn Profile\nОбразование\n•Бакалавриат: Национальный исследовательский технологический университет МИСиС 2018-2023\nНаправление: Прикладная математика GPA: 4.1/5.0\nТема диплома: Динамическая оптимизация инвестиционного портфеля с Reinforcement Learning\nОпыт работы\n•Pheon Digital Twins Дек 2022 - Сейчас\nNLP Engineer International', metadata={'page': 0, 'source': 'data/Сергей Братчиков, CV.pdf'}),
  0.4115613710445648),
 (Document(page_content='Технические навыки и интересы\nЯзыки: Python, Java, Kotlin, C++, SQL\nИнструменты разработки : Pycharm, Docker, Git\nФреймворки : Torch, Transformers, W&B, ONNX, FastAPI\nCloud/Databases : AWS, GC, Postgres\nSoft Skills : Agile, Понимание принципов комплаенс, Инициативность\nОбласти интереса : NLP Research, Multimodality, Transformers, RL in finance\nДостижения\n•С

## Retrievers

In [194]:
retriever = MultiQueryRetriever.from_llm(vectorstore.as_retriever(search_kwargs={"k": 3}), llm)

In [195]:
retriever.get_relevant_documents("Места работы")

[Document(page_content='Я принимаю условия соглашения и даю своё согласие на обработку персональных данных в соответствии \n                            с Политикой конфиденциальности\n                             Федерального Государственного Унитарного Предприятия «Международное информационное агентство «Россия сегодня», \n                            расположенного по адресу: Россия, 119021, г. Москва, Зубовский бульвар, д. 4.', metadata={'description': 'Правительство Японии сожалеет о решении России приостановить налоговые соглашения с рядом стран и направило протест по дипломатическим каналам, потребовав... РИА Новости, 09.08.2023', 'language': 'ru', 'source': 'http://ria.ru/20230809/yaponiya-1888975642.html', 'title': 'Япония выразила протест против решения России о налоговом соглашении - РИА Новости, 09.08.2023'}),
 Document(page_content='Почта\n                        \n                            Пароль\n                        Войти\n                        Восстановить пароль\

## ConvRetrievalChain

In [209]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [210]:
conv_chain = ConversationalRetrievalChain.from_llm(llm, retriever, memory=memory)
conv_chain

ConversationalRetrievalChain(memory=ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[]), output_key=None, input_key=None, return_messages=True, human_prefix='Human', ai_prefix='AI', memory_key='chat_history'), callbacks=None, callback_manager=None, verbose=False, tags=None, metadata=None, combine_docs_chain=StuffDocumentsChain(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, metadata=None, input_key='input_documents', output_key='output_text', llm_chain=LLMChain(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, metadata=None, prompt=ChatPromptTemplate(input_variables=['context', 'question'], output_parser=None, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], output_parser=None, partial_variables={}, template="Use the following pieces of context to answer the users question. \nIf you don't know the answer, just say that you don't know, don't try to mak

In [406]:
conv_chain({"question": "Что ты знаешь о Сергее Братчикове?"})['answer']

KeyboardInterrupt: 

## Tool based search

In [18]:
custom_tool = Tool(name='save_note', func=lambda x: {'action': print(x)}, description='Useful for when you need to save note or reminder for user request')

In [19]:
retr_tool = create_retriever_tool(
    vectorstore.as_retriever(search_type='similarity', search_kwargs={'k': 5}),
    "search_in_docs",
    "Useful for when you need to answer question you don't know or you cant find, about people, documents and other.\nSearches and returns documents regarding to query."
)
# wiki_tool = WikipediaQueryRun(api_wrapper=wiki_api)
ddg_tool = DuckDuckGoSearchRun(api_wrapper=ddg_api)
# tools = [retr_tool, wiki_tool]
tools = [retr_tool]
tools

[Tool(name='search_in_docs', description="Useful for when you need to answer question you don't know or you cant find, about people, documents and other.\nSearches and returns documents regarding to query.", args_schema=None, return_direct=False, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, handle_tool_error=False, func=<bound method BaseRetriever.get_relevant_documents of VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], metadata=None, vectorstore=<langchain.vectorstores.chroma.Chroma object at 0x7f5a77343340>, search_type='similarity', search_kwargs={'k': 5})>, coroutine=None)]

In [20]:
from langchain.agents.openai_functions_agent.agent_token_buffer_memory import AgentTokenBufferMemory

agent_memory = AgentTokenBufferMemory(memory_key="chat_history",
                                      llm=llm,
                                      max_token_limit=3000)
agent_memory.clear()

In [21]:
PREFIX = """Answer the following questions as best you can. Use russian language. You have access to the following tools:"""
mrkl = initialize_agent(
    tools, llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    # agent_kwargs={'prefix': PREFIX},
    verbose=True,
    memory=agent_memory,
    return_intermediate_steps=True
)

In [23]:
mrkl_result = mrkl({"input": "Кто такой Братчиков Сергей, какой у него опыт?"})
mrkl_result['output'], mrkl_result['intermediate_steps']



> Entering new AgentExecutor chain...

Invoking: `search_in_docs` with `{'query': 'Братчиков Сергей'}`


[Document(page_content='Братчиков Сергей Станиславович ♂phone+79859856098\nРоль: ML Engineer /envel⌢pehivaze.me@gmail.com\n31.10.2000 (22 года) /githubGitHub Profile\n/linkedinLinkedIn Profile\nОбразование\n•Бакалавриат: Национальный исследовательский технологический университет МИСиС 2018-2023\nНаправление: Прикладная математика GPA: 4.1/5.0\nТема диплома: Динамическая оптимизация инвестиционного портфеля с Reinforcement Learning\nОпыт работы\n•Pheon Digital Twins Дек 2022 - Сейчас\nNLP Engineer International', metadata={'page': 0, 'source': 'data/Сергей Братчиков, CV.pdf'}), Document(page_content='Junior ML Engineer Москва\n– Создание системы для асинхронного анализа мультистраничных сканированных документов\n– Создание OCR пайплайнов с алгоритмами на основе OpenCV, TesseractOCR и EasyOCR\n– Решение NLP задач классификации, синтаксического анализа и графов знаний\n– Участие в ра

('Братчиков Сергей Станиславович - ML инженер с опытом работы. Он имеет следующий опыт и достижения:\n\n- Опыт работы в компании Pheon Digital Twins с декабря 2022 года.\n- Опыт работы в Sber Robotics Lab в качестве исследовательского стажера с июня 2020 года до августа 2020 года.\n- Участие в различных хакатонах, где занимал первые места в командах по разработке ML-сервисов, решению NLP задач и оптимизации упаковки товаров.\n- Победитель трека "Сравнение и автоматическая оценка образовательных программ вузов" на мероприятии Цифровой прорыв (Сезон AI) в 2021 году.\n- Победитель трека "Оптимизация упаковки 3D товаров на палете" на мероприятии Magnetic Code Hack в 2022 году.\n- Занял 78-е место на финальном лидерборде MTS ML CUP в 2022 году в задаче классификации последовательностей действий пользователей.\n- Финалист и занял 7-е место на лидерборде Lenta HackTheCart в 2021 году в задаче рекомендации товаров.\n- Участие в проекте "Нейростаниславский", где создавал сервис с GPT моделью в 

In [266]:
agent_executor = create_conversational_retrieval_agent(llm, tools, verbose=True)

In [267]:
result = agent_executor({"input": "Владимир Путин и как он связан с Сергеем Братчиковым"})
result



> Entering new AgentExecutor chain...

Invoking: `Wikipedia` with `{'query': 'Владимир Путин'}`


Page: Vladimir Putin
Summary: Vladimir Vladimirovich Putin (born 7 October 1952) is a Russian politician and former intelligence officer serving as the current president of Russia. Putin has served continuously as president or prime minister since 1999: as prime minister from 1999 to 2000 and from 2008 to 2012, and as president from 2000 to 2008 and since 2012.Putin worked as a KGB foreign intelligence officer for 16 years, rising to the rank of lieutenant colonel before resigning in 1991 to begin a political career in Saint Petersburg. He moved to Moscow in 1996 to join the administration of President Boris Yeltsin. He briefly served as director of the Federal Security Service (FSB) and secretary of the Security Council of Russia, before being appointed prime minister in August 1999. After the resignation of Yeltsin, Putin became acting president and, less than four months later, was el

{'input': 'Владимир Путин и как он связан с Сергеем Братчиковым',
 'chat_history': [HumanMessage(content='Владимир Путин и как он связан с Сергеем Братчиковым', additional_kwargs={}, example=False),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'Wikipedia', 'arguments': '{\n  "query": "Владимир Путин"\n}'}}, example=False),
  FunctionMessage(content="Page: Vladimir Putin\nSummary: Vladimir Vladimirovich Putin (born 7 October 1952) is a Russian politician and former intelligence officer serving as the current president of Russia. Putin has served continuously as president or prime minister since 1999: as prime minister from 1999 to 2000 and from 2008 to 2012, and as president from 2000 to 2008 and since 2012.Putin worked as a KGB foreign intelligence officer for 16 years, rising to the rank of lieutenant colonel before resigning in 1991 to begin a political career in Saint Petersburg. He moved to Moscow in 1996 to join the administration of President Boris Yeltsin